# DS775 Python Needs from DS710

## General Stuff

* Please cover at least a tiny bit about Jupyter notebooks since, for better or worse, they're ubiquitous in data science and we utilize them in other courses (DS775 at least)
    * Students need to understand about kernels and state.  We still get students executing cells out of order and wondering why they get errors
    * Debugging in Jupyter notebooks.  It would be super slick if they knew how to save their notebook as a .py file and open it in Spyder to apply debugging tools.  Or to develop code in Spyder first before moving it to a notebook.  Jupyter Lab also has gui assisted debugging available, but that requires installation of a different kernel.

* Debugging in general
    * even just adding print statements seems beyond some of our students

* How to study and understand code instead of copying and randomly tweaking
    * learn to work through code line by line to really grok it

* Problem solving - make a point of giving problems that require original code and synthesis of ideas

* Formatting output.  We're moving toward f-strings.  

Now we'll list some particular issues in some of the units we cover.  The complete notebooks for these lessons and the corresponding homework assignments may be found on Github at https://github.com/DataScienceUWL/DS775v2.

## Lesson 01: Linear Programming and Pyomo

* String formatting for formatting output (we are trying to standardize all of ours to f-strings)

## Lesson 02: Sensitivity Analysis and Abstract Modeling

* Data Structures
    * Dictionaries
    * Nested dictionaries
    * Lists and nested lists
    * Pandas dataframes
    
A primary focus in this lesson is getting the students to move from very concrete linear programs with hard-wired coefficients to more abstract formulations that separate the model and the data so they can generalize to larger and different problems.

We want them to move from the model in the left column to the one in the right column:

<table class="tleft">
    <tr>
        <td><p style="padding-right: 120px;"> <b>Concrete Model</b></p></td>
        <td><p style="padding-right: 150px;"> <b>Abstract Model</b></p></td>
    </tr>
<tr>
<td>
Maximize $Z = 3 d + 5 w$
</td>
<td>
Maximize $ Z = \displaystyle \sum_{pr \in Pr} c_{pr} x_{pr}$
<td>
</tr>
<tr>
<td>
Subject to:

$
\begin{array}{ccccc}
 d &   &    & \leq & 4 \\
   &   & 2w & \leq & 12 \\
3d & + & 2w & \leq & 18
\end{array}
$
</td>
<td>
Subject to:

$ \displaystyle \sum_{pr \in Pr} h_{pl,pr} x_{pr} \leq a_{pl},$
$\mbox{ for each } pl \in Pl$
<td>
</tr>
<tr>
<td>
$d \geq 0$, $w \geq 0$
</td>
<td>
$ x_{pr} \geq 0, \mbox{ for each } pr \in Pr$ 
<td>
</tr>   
</table>

Code for the left column, concrete approach is here:

In [0]:
# Unfold to see the Pyomo solution with individual decision variables
from pyomo.environ import *

# Concrete Model
model = ConcreteModel(name="Wyndor")

# Decision Variables
model.doors = Var(domain=Reals)
model.windows = Var(domain=Reals)

# Objective
model.profit = Objective(expr=3.0 * model.doors + 5.0 * model.windows,
                         sense=maximize)

# Constraints
model.Plant1 = Constraint(expr=model.doors <= 4)
model.Plant2 = Constraint(expr=2.0 * model.windows <= 12)
model.Plant3 = Constraint(expr=3.0 * model.doors + 2.0 * model.windows <= 18)
model.dgeq0 = Constraint(expr=model.doors >= 0)
model.wgeq0 = Constraint(expr=model.windows >= 0)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

#display(model)

# display solution
import babel.numbers as numbers  # needed to display as currency
print("Maximum Profit = ",
      numbers.format_currency(1000 * model.profit(), 'USD', locale='en_US'))
print("Batches of Doors = {}".format(model.doors()))
print("Batches of Windows = {}".format(model.windows()))

Code for the right column, abstract approach is here:

In [0]:
# Unfold for code
from pyomo.environ import *
products = ['Doors', 'Windows']
plants = ['Plant1', 'Plant2', 'Plant3']
profit_rate = {'Doors': 3, 'Windows': 5}
hours_available = {'Plant1': 4, 'Plant2': 12, 'Plant3': 18}
hours_per_batch = {
    'Plant1': {
        'Doors': 1,
        'Windows': 0
    },
    'Plant2': {
        'Doors': 0,
        'Windows': 2
    },
    'Plant3': {
        'Doors': 3,
        'Windows': 2
    }
}

#Concrete Model
model = ConcreteModel()

#Decision Variables
model.weekly_prod = Var(products, domain=NonNegativeReals)

#Objective
model.profit = Objective(expr=sum(profit_rate[pr] * model.weekly_prod[pr]
                               for pr in products),
                      sense=maximize)

model.capacity = ConstraintList()
for pl in plants:
    model.capacity.add(
        sum(hours_per_batch[pl][pr] * model.weekly_prod[pr]
            for pr in products) <= hours_available[pl])

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# display solution
import babel.numbers as numbers  # needed to display as currency
print("Maximum Profit = ",
      numbers.format_currency(1000 * model.profit(), 'USD', locale='en_US'))

for j in products:
    print("Batches of " + j + " = {}".format(model.weekly_prod[j]()))

The abstract approach is much more difficult for students as they need a solid mastery of lists, dictionaries, and loops in addition to plenty of practice putting it together.

I think that some may also struggle with the general idea of using lists as index sets.

Pandas dataframes are used to tabulate and display results in many of our problems.

## Lesson 03: Transportation Problems and More Abstract Formulations

* Slicing n-dim arrays/lists/dictionaries, summing across dimensions

Note: These examples use real data from homework problems. Do not use these exact problems.

In [3]:
#given data structures like these
warehouses = ['wA', 'wB', 'wC']
stores = ['sA', 'sB', 'sC', 'sD', 'sE', 'sF', 'sG', 'sH', 'sI', 'sJ', 'sK', 'sL', 'sM', 'sN', 'sO', 'sP', 'sQ', 'sR', 'sS', 'sT']

cost_dic =   {('wA', 'sA'): 7, ('wA', 'sB'): 8, ('wA', 'sC'): 13, ('wA', 'sD'): 12, ('wA', 'sE'): 7, ('wA', 'sF'): 6, ('wA', 'sG'): 9, ('wA', 'sH'): 10, ('wA', 'sI'): 14, ('wA', 'sJ'): 13, ('wB', 'sG'): 8, ('wB', 'sH'): 6, ('wB', 'sI'): 12, ('wB', 'sJ'): 11, ('wB', 'sK'): 10, ('wB', 'sL'): 10, ('wB', 'sM'): 7, ('wB', 'sN'): 14, ('wB', 'sO'): 11, ('wB', 'sP'): 5, ('wC', 'sL'): 7, ('wC', 'sM'): 14, ('wC', 'sN'): 13, ('wC', 'sO'): 7, ('wC', 'sP'): 8, ('wC', 'sQ'): 15, ('wC', 'sR'): 12, ('wC', 'sS'): 11, ('wC', 'sT'): 11}

#be able to conditionally sum like this
total = 0
for w in warehouses:
    total += sum(cost_dic[(w, s)] for s in stores if (w,s) in cost_dic)

f"Total: {total}"

'Total: 291'

In [11]:
import pandas as pd
#given data structures like these:
all_routes = [('rail', 's1', 'm1'), ('rail', 's1', 'm2'), ('rail', 's1', 'm3'), ('rail', 's1', 'm4'), ('rail', 's1', 'm5'),
               ('rail', 's2', 'm1'), ('rail', 's2', 'm2'), ('rail', 's2', 'm3'), ('rail', 's2', 'm4'), ('rail', 's2', 'm5'),
               ('rail', 's3', 'm1'), ('rail', 's3', 'm2'), ('rail', 's3', 'm3'), ('rail', 's3', 'm4'), ('rail', 's3', 'm5'),
          ('ship','s1', 'm1'), ('ship','s1', 'm2'), ('ship','s1', 'm3'),  ('ship','s1', 'm5'),
          ('ship','s2', 'm1'), ('ship','s2', 'm2'), ('ship','s2', 'm3'), ('ship','s2', 'm4'), ('ship','s2', 'm5'),
          ('ship','s3', 'm2'), ('ship','s3', 'm3'), ('ship','s3', 'm4'), ('ship', 's3', 'm5')
        ]
costs = [61,72,45,55,66,69,78,60,49,56,59,66,63,61,47,
                                    58.5, 68.3, 47.8, 63.5, 65.3, 74.8, 55.0, 49.0, 57.5, 61.3, 63.5, 58.8, 50.0]

routetypes = ['rail', 'ship']
sources = ['s1', 's2', 's3']
markets = ['m1', 'm2', 'm3', 'm4', 'm5']

#be able to zip things like this
all_route_costs = dict(zip(all_routes, costs))

#be able to subset like this:
ship_routes = [(t,s,m) for (t,s,m) in all_route_costs if t == 'ship']

#be able to generate data frames like this:
for t in routetypes:
    print(f"Shipment costs from each source to each market for the {t} method:")
    schedule = pd.DataFrame(0, index=sources, columns=markets)
    for (t2,s,m) in all_routes:
        if t == t2:
            schedule.loc[s,m] = all_route_costs[t,s,m]

    print(schedule)     

Shipment costs from each source to each market for the rail method:
    m1  m2  m3  m4  m5
s1  61  72  45  55  66
s2  69  78  60  49  56
s3  59  66  63  61  47
Shipment costs from each source to each market for the ship method:
      m1    m2    m3    m4    m5
s1  58.5  68.3  47.8   0.0  63.5
s2  65.3  74.8  55.0  49.0  57.5
s3   0.0  61.3  63.5  58.8  50.0


### 3 dimensional data

We do some transportation problems with 3 dimensional data structures and many students do just fine, but many are confounded by things like this:

To find the cost of shipping the products we have to sum the cost per unit times the number of units over all the elements in the three dimensional array, like this:

To find the cost of shipping the products we have to sum the cost per unit times the number of units over all the elements in the three dimensional array, like this:

```
sum(cost[p,s,c] * model.transp[p,s,c] for p in products for s in suppliers for c in customers)
```

A supply constraint means that total amount of each product shipped from each supplier must match the supply available.  

```
for p in products:
    for s in suppliers:
        sum( model.transp[p,s,c] for c in customers) == supply[p,s] )
```
In the picture above this corresponds to summing each row of the stacked 2D arrays to make sure it matches the supplied amount.

If limited capacity is available for shipping from each supplier to each customer we have to add up the total amount of all products to be sure it isn't too large:
```
for s in supplier:
    for c in customer:
        sum( model.transp[p,s,c] for p in products ) <= capacity )
```

**When they try to write their own code they get confused about which variables to use for slices and which to sum over.**  Those who just mimic code without understanding can really make a mess with these constructions.


### Recognizing when to write functions.

In this homework we have them solve the same transportation problem three times with different shipping cost data.  I'm always amazed by how many cut and paste all of their code three times instead of writing a function and calling it three times.

## Lesson 04: Quadratic Programming and Local Optimization

### Numpy arrays and lambda functions

Here is a small bit of code that I think some don't understand very well:
```
# plot p(x) on [-10,10]
x = np.linspace(-10,10,201)
p = lambda x:x**4 + 2*x**3 + 3*x**2 + 2*x + 1
fig = plt.figure(figsize=(4,3.5))
plt.plot(x,p(x));
plt.xlabel('x');
plt.ylabel('y');
```
A few seem to have trouble just reading documentation.  I don't explain linspace from numpy, but expect them to just look it up (most, but not all, are OK at this).  I also always have a couple that try `x**4` when `x` is a list instead of a numpy array.  **Practice with Numpy and vectorization would be good.**

### Passing functions to functions and variable numbers of arguments (*args, *kwargs)

When they're learning about optimization they need to pass their fitness functions to optimization routines from scipy.optimize and elsewhere.  Often times these functions require additional arguments.  I think some are pretty confused about how this works.  Here is an example:

```
def neg_log_loss( coef, *args):
    b0 = coef[0]
    b1 = coef[1]
    x = args[0]
    y = args[1]
    p = 1.0/(1.0 + np.exp(-(b0 + b1*x)))
    ll = sum( y*np.log(p)+(1-y)*np.log(1-p) )
    return(-ll) # here's the minus sign!
    
from scipy.optimize import minimize
result = minimize(neg_log_loss,[0,0],args=(x_hours,y_passed))
```

### Parsing and Understanding Code

We give them this code for a local search algorithm for the traveling salesman problem and later ask them to build similar code to solve discrete optimization problems.  

```
def random_reversals(dist_mat, max_no_improve):
    num_cities = len(dist_mat)
    # starts from a random tour
    current_tour = np.random.permutation(np.arange(num_cities))
    current_dist = tour_distance(current_tour, dist_mat)
    best_tour = current_tour
    best_dist = current_dist

    # stop search if no better tour is found within max_no_improve iterations, can increase to eliminate crossovers
    num_moves_no_improve = 0
    iterations = 0
    while (num_moves_no_improve < max_no_improve):
        num_moves_no_improve += 1
        iterations += 1  # just for tracking
        new_tour = sub_tour_reversal(current_tour)
        new_dist = tour_distance(new_tour, dist_mat)
        if new_dist < current_dist:
            num_moves_no_improve = 0
            current_tour = new_tour
            current_dist = new_dist
            if current_dist < best_dist:  # not really needed since current_tour will be best
                best_tour = current_tour  # but we'll use this in the next lesson
                best_dist = current_dist
    return best_tour, best_dist, iterations
```

We build up to this code starting from pseudocode, but it's evident, based on some of the submissions we see, that some can't really break down and understand the code.  Learning how to study and analyze code is something they can work on.

## Lesson 05: Global Optimization and Metaheuristics

In this lesson we include simulated annealing for discrete optimization.  In addition to providing code similar to the local search code shown above in Lesson 04, we also make use of the simanneal package (https://github.com/perrygeo/simanneal).  To use it students have to write init and two other methods in a class.  Like this (we provide this example in the lesson):

```
from simanneal import Annealer

class TravellingSalesmanProblem(Annealer):

    # pass extra data (the distance matrix) into the constructor
    def __init__(self, state, distance_matrix):
        self.distance_matrix = distance_matrix
        super(TravellingSalesmanProblem, self).__init__(state)  # important!

    def move(self):
        """Reverse random segment"""
        tour = self.state
        n = len(tour)
        i, j = np.sort(np.random.choice(n, 2, replace=False))
        swapped = np.concatenate((tour[0:i], tour[j:-n + i - 1:-1],tour[j + 1:n]))
        self.state = swapped.tolist()
    
    def energy(self):
        """Compute tour distance"""
        dist_mat = self.distance_matrix
        tour = self.state
        distance = dist_mat[tour[-1]][tour[0]]
        for gene1, gene2 in zip(tour[0:-1], tour[1:]):
            distance += dist_mat[gene1][gene2]
        return distance


# load data (this may have to be adapted for different problems)
with open("data/HillierTSP.json", "r") as tsp_data:
    tsp = json.load(tsp_data)
distance_matrix = tsp["DistanceMatrix"]
init_tour = np.random.permutation(np.arange(len(distance_matrix))).tolist()

tsp = TravellingSalesmanProblem(init_tour, distance_matrix)
tsp.set_schedule(tsp.auto(minutes=.2)) #set approximate time to find results
# since our state is just a list, slice is the fastest way to copy
tsp.copy_strategy = "slice" #"method"
best_tour, best_dist = tsp.anneal()

best_dist
```

This ends up being a big lift for some students who I think just haven't seen much of this.  They seem to not "get" the whole paradigm of storing state in the object and using methods to modify the state.  In the homework they run into problems using simanneal for a knapsack problem.  The solution for that problem is shown below.

In [4]:
# generate random weights and values for a knapsack problem
import numpy as np
num_items = 20
np.random.seed(seed=123)
values = np.random.randint(low=5, high=50, size=num_items)
weights = np.random.randint(low=1, high=10, size=num_items)
max_weight = 50
np.random.seed() # use system clock to reset the seed so future random numbers will appear random

In [0]:
# unfold code
from simanneal import Annealer

class knapsack_problem(Annealer):
    # pass extra data (the distance matrix) into the constructor
    def __init__(self, state, values, weights, max_weight):
        self.weights = weights
        self.values = values
        self.max_weight = max_weight
        super(knapsack_problem, self).__init__(state)  # important!

    def move(self):
        # copy current items
        x = self.state.copy()
        # toggle one item
        bit_to_flip = np.random.randint(len(x))
        x[bit_to_flip] = not x[bit_to_flip]
        # accept if weight <= max weight
        if sum(self.weights[x]) <= self.max_weight:
            self.state = x

    def energy(self):
        # add up values of selected items, return negative for maximization
        return -sum( self.values[ self.state] )

items = np.zeros(20, dtype = bool)

ksp = knapsack_problem(items, values, weights, max_weight)
ksp.set_schedule(ksp.auto(minutes=.1))
ksp.copy_strategy = "method"
items, value = ksp.anneal()

# using f-strings (https://www.geeksforgeeks.org/formatted-string-literals-f-strings-python/)
print(f"knapsack: {items}\nvalue: {-value}")

One of the main issues that students have (and we can do a better job here) is that they don't understand the difference between
```
x = self.state
```

and 

```
x = self.state.copy()
```

in the move method.  Those who don't use copy() end up accepting infeasible states by accident because they unintentionally modify self.state.

## Lesson 06: Integer Programming

The python code is similar to that in Lessons 2 and 3 so the issues are similar.

## Lesson 07: Constraint Programming

The issues in this section are mostly conceptual.  However, we use the CP-SAT constraint programming solver which is part of Google's ortools package (https://developers.google.com/optimization/cp/cp_solver).  Weaker students struggle here because of the lack of examples they can mimic and inability to read documentation effectively.  I'm not sure what DS710 can do here except perhaps require students to sometimes figure things out from documentation alone perhaps.

They're also hampered by having to understand the use of a class to pass to the solver for callbacks.  There is an example at the link above.  Seeing more OOP in DS710 could help with this.

## Lesson 08: Simulation

* Looping versus vectorization
    * e.g. rolling a die 1000 times and counting the number of 4's, can be done in one or two lines with numpy, but noobs wanna loop.
* random sampling and random number seeds
    * we use distributions from numpy.random and some don't seem to grasp the use of seeds for reproducibility
* estimating probabilities by relative frequency in a random sample
    * I think we did something wrong in DS705 when students don't understand this :(

## Lesson 09: Decision Analysis

* not much Python here.

## Lesson 10 and 11:  Recommender Systems

* We make heavy use of Pandas dataframes in here including selecting and appending columns, filtering rows, sorting, and merging dataframes.  
* There is a fair amount of text processing here.  For example taking movie descriptions and removing stop words, punctuation, extra spaces, etc.
* For examples of some of this see: https://github.com/DataScienceUWL/DS775v2/blob/master/Lessons/Lesson%2010%20Presentation%20-%20ResSys%201/Content%20Based%20Recommenders.ipynb (fwiw, this was provided with a text we adopted and it isn't fantastically written python)
* We also make use of the machine learning package `sklearn` here and in a course project.  For most it's the first time they've seen it.  It's an important package and perhaps worthy